## 실습 문제 1: ToDo List

할 일(Task) 목록을 파일에 저장하고, 로드할 수 있는 ToDoList 클래스를 작성하세요.
이 클래스는 할 일을 추가/완료 표시/목록 출력 기능과 더불어, 파일로부터 할 일 목록을 읽어오고 저장하는 기능을 가져야 합니다.
할 일은 내부 리스트(tasks)에 사전(dict) 형태로 저장하며, 각 사전은 {"description": str, "completed": bool} 구조를 가집니다.

**Methods**
* __init__(): 내부 리스트 tasks를 빈 상태로 초기화합니다.

* add_task(description: str): 새 할 일을 {"description": description, "completed": False} 형태로 tasks에 추가합니다.

* complete_task(index: int): tasks[index]["completed"]를 True로 변경합니다. 유효하지 않은 인덱스면 IndexError를 발생시킵니다.

* list_tasks(): 현재 tasks를 순회하며 다음과 같은 형식으로 출력합니다.

    [ ] Buy milk<br>
    [x] Write report

* load_tasks(file_path: str): 지정된 file_path에 저장된 JSON 파일을 읽어와 tasks를 덮어씁니다. 파일이 없거나 형식이 잘못되면 빈 리스트를 유지합니다.

* save_tasks(file_path: str): 현재 tasks를 JSON 형식으로 직렬화하여 file_path에 저장합니다. 기존 파일이 있으면 덮어씁니다.

    파일 포맷:
```
[
  {"description": "Buy milk", "completed": false},
  {"description": "Write report", "completed": true}
]
```



pandas 를 이용해서 실습문제 4번 풀어 보기

In [4]:
import json

class ToDoList:
  def __init__(self):
    self.tasks = []

  def add_task(self, desc: str):
    self.tasks.append({"descripttion": desc, "completed": False})

  def complete_task(self, index: int):
    if 0 <= index < len(self.tasks):
      self.tasks[index]["completed"] = True
    else:
      raise IndexError("Invalid task index")

  def list_tasks(self):
    for task in self.tasks:
      check = "[x]" if task["completed"] else "[]"
      print (f"{check} {task['description']}")

  def save_tasks(self, file_path: str):
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(self.tasks, f, ensure_ascii=False, indent=2)

  def load_tasks(self, file_path: str):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                self.tasks = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            self.tasks = []      

In [7]:
import os
import json

input_data_tasks = [
    [("add", "Buy milk"), ("add", "Write report"), ("complete", 1)],
    [("add", "Read book"), ("complete", 0), ("add", "Go jogging")]
]
print(input_data_tasks)

[[('add', 'Buy milk'), ('add', 'Write report'), ('complete', 1)], [('add', 'Read book'), ('complete', 0), ('add', 'Go jogging')]]


In [5]:
import os
import json

input_data_tasks = [
    [("add", "Buy milk"), ("add", "Write report"), ("complete", 1)],
    [("add", "Read book"), ("complete", 0), ("add", "Go jogging")]
]

expected_task_states = [
    [
        {"description": "Buy milk",     "completed": False},
        {"description": "Write report", "completed": True}
    ],
    [
        {"description": "Read book",    "completed": True},
        {"description": "Go jogging",   "completed": False}
    ]
]

for actions, expected in zip(input_data_tasks, expected_task_states):
    todo = ToDoList()
    for action, value in actions:
        if action == "add":
            todo.add_task(value)
        elif action == "complete":
            todo.complete_task(value)
    assert todo.tasks == expected, f"Expected {expected}, but got {todo.tasks}"


path = "test_tasks.json"


if os.path.exists(path):
    os.remove(path)

orig = ToDoList()
orig.add_task("Pay bills")
orig.complete_task(0)
orig.save_tasks(path)

loaded = ToDoList()
loaded.load_tasks(path)
assert loaded.tasks == orig.tasks, f"After load, expected {orig.tasks}, but got {loaded.tasks}"

os.remove(path)

AssertionError: Expected [{'description': 'Buy milk', 'completed': False}, {'description': 'Write report', 'completed': True}], but got [{'descripttion': 'Buy milk', 'completed': False}, {'descripttion': 'Write report', 'completed': True}]

## 실습 문제 2: 은행 계좌 관리
이 문제는 은행의 계좌를 관리하는 시스템을 클래스를 사용하여 구현하는 것입니다. BankService 클래스는 은행을 나타내며, 내부적으로 딕셔너리를 사용하여 자신의 은행에서 개설한 BankAccount 객체들을 관리합니다. 계좌는 BankAccount 클래스로 정의되며, 계좌 번호를 키로 하여 계좌를 추가, 삭제하고, 계좌 정보를 조회할 수 있습니다. 또한, 계좌 정보를 CSV 파일로 저장하고 불러오는 기능을 포함합니다.

### BankService 클래스

**Properties:**
* csv_file: 계좌 정보를 저장할 CSV 파일 경로 (기본값: 'accounts.csv')
* accounts: 계좌 번호를 키로 하고, BankAccount 객체를 값으로 가지는 딕셔너리

**Methods:**
* __init__(self, csv_file='accounts.csv'): CSV 파일 경로를 받아 초기화하고, 파일에서 계좌 정보를 불러옵니다.
* load_from_csv(self): CSV 파일에서 계좌 정보를 불러와 accounts 딕셔너리에 저장합니다.
* save_to_csv(self): accounts 딕셔너리의 계좌 정보를 CSV 파일에 저장합니다.
* add_account(self, account_number, initial_balance=0): 새로운 계좌를 생성하여 딕셔너리에 추가하고, CSV 파일에 저장합니다. 이미 존재하는 계좌 번호일 경우 오류 메시지를 출력합니다.
* remove_account(self, account_number): 지정된 계좌 번호의 계좌를 딕셔너리에서 삭제하고, CSV 파일에 저장합니다. 계좌가 존재하지 않을 경우 오류 메시지를 출력합니다.
* get_account(self, account_number): 지정된 계좌 번호의 BankAccount 객체를 반환합니다. 계좌가 존재하지 않을 경우 오류 메시지를 출력합니다.
* deposit_to_account(self, account_number, amount): 지정된 계좌에 입금을 수행합니다.
* withdraw_from_account(self, account_number, amount): 지정된 계좌에서 출금을 수행합니다.
* get_all_accounts(self): 모든 계좌의 계좌 번호와 잔액을 출력합니다.


### BankAccount 클래스

**Properties:**
* account_number: 계좌 번호 (문자열)
* balance: 잔액 (실수형, 기본값 0)
* BankService: BankService 클래스의 인스턴스

**Methods:**
* __init__(self, bank_service, account_number, initial_balance=0): BankService 객체, 계좌 번호, 초기 잔액을 받아 계좌를 초기화합니다.
* deposit(self, amount): 지정된 금액을 입금하고, 입금 후 잔액을 반환합니다. 입금 금액이 0 이하일 경우 오류 메시지를 출력합니다。
* withdraw(self, amount): 지정된 금액을 출금하고, 출금 후 잔액을 반환합니다. 출금 금액이 잔액보다 크거나 0 이하일 경우 오류 메시지를 출력합니다.
* get_balance(self): 현재 잔액을 반환합니다.
* get_account_number(self): 계좌 번호를 반환합니다.

In [ ]:
import csv

class BankService:
  def __init__(self, csv_file='accounts.csv'):
    self.csv_file = csv_file
    self.accounts = {}
    self.load_from_csv()

  def load_from_csv(self):
    pass

  # Implement your code

class BankAccount:
  def __init__(self, bank_service, account_number, initial_balance=0):
    self.bank_service = bank_service
    self.account_number = account_number
    self.balance = initial_balance

  def deposit(self, amount):
    pass

  # Implement your code

In [ ]:
input_data_scenarios = [
    [
        ("add", "111-222", 1000),
        ("add", "333-444", 500),
        ("deposit", "111-222", 200),
        ("withdraw", "333-444", 100),
    ],
    [
        ("add", "777-888", 2000),
        ("withdraw", "777-888", 1500),
        ("add", "555-666", 0),
        ("deposit", "555-666", 300)
    ]
]

expected_account_states = [
    {"111-222": 1200, "333-444": 400},
    {"777-888": 500, "555-666": 300}
]

for actions, expected_state in zip(input_data_scenarios, expected_account_states):
    temp_csv_file = 'test_scenario_accounts.csv'
    if os.path.exists(temp_csv_file):
        os.remove(temp_csv_file)

    bank_service = BankService(csv_file=temp_csv_file)
    for action, *params in actions:
        if action == "add":
            bank_service.add_account(params[0], params[1])
        elif action == "deposit":
            bank_service.deposit_to_account(params[0], params[1])
        elif action == "withdraw":
            bank_service.withdraw_from_account(params[0], params[1])
        elif action == "remove":
            bank_service.remove_account(params[0])

    current_state = {num: acc.get_balance() for num, acc in bank_service.accounts.items()}
    assert current_state == expected_state, f"예상 결과: {expected_state}, 실제 결과: {current_state}"
    print(f"시나리오 통과. 최종 상태: {current_state}")

    if os.path.exists(temp_csv_file):
        os.remove(temp_csv_file)


In [ ]:
test_csv_path = "test_bank_accounts.csv"

if os.path.exists(test_csv_path):
    os.remove(test_csv_path)

original_service = BankService(csv_file=test_csv_path)
original_service.add_account("100-A", 5000)
original_service.add_account("200-B", 10000)
original_service.deposit_to_account("100-A", 500)
original_service.get_all_accounts()

loaded_service = BankService(csv_file=test_csv_path)
loaded_service.get_all_accounts()

original_state = {num: acc.get_balance() for num, acc in original_service.accounts.items()}
loaded_state = {num: acc.get_balance() for num, acc in loaded_service.accounts.items()}

assert loaded_state == original_state, f"파일 로드 후, 예상 결과: {original_state}, 실제 결과: {loaded_state}"

print("\nCSV 저장 및 불러오기 기능이 정상적으로 동작합니다.")

if os.path.exists(test_csv_path):
    os.remove(test_csv_path)

## 실습 문제 3: 파일실습

파일에서 숫자 데이터를 처리하고 합계, 평균, 최소값, 최대값 및 중앙값과 같은 기본 통계를 계산하는 메서드를 제공하는 FileProcessor 클래스를 작성하세요.

파일: Numbers.txt (https://drive.google.com/file/d/1UgrWNIimGqBE48is_LEhBEZVW5DNCk2t/view?usp=sharing)

### FileProcessor 클래스

* 생성자: FileProcessor 객체를 초기화합니다. 파일 이름을 입력으로 받아 저장하고, 이후에 파일에서 읽어온 숫자 데이터를 저장할 빈 리스트 self.data를 초기화합니다.
힌트: 이 메서드는 파일을 읽고 처리할 준비를 합니다. 파일 경로는 저장되지만, 데이터는 다른 메서드에서 요청할 때까지 로드되지 않습니다.

* read_file(self): 파일에서 데이터를 읽습니다. 파일을 읽기 모드로 열고, 각 줄에서 불필요한 문자를 제거한 후 데이터를 실수로 변환합니다. 이 처리된 데이터를 self.data에 저장합니다.
힌트: 이 메서드를 사용하여 필요한 경우에만 데이터를 지연 로드합니다. FileNotFoundError 및 ValueError와 같은 파일 관련 오류를 처리하여 파일 읽기가 견고하게 이루어지도록 합니다.

* sum(self): 숫자 데이터의 합계를 계산합니다. 결과를 소수점 한 자리까지 반올림하여 반환합니다.
힌트: 계산을 수행하기 전에 파일이 읽혔는지 확인하세요. 내장된 sum() 함수를 사용하여 코드를 간소화할 수 있습니다. 빈 파일이 있을 수 있으므로 _read_file이 성공적으로 호출된 경우에만 이 메서드가 제대로 작동합니다.

* average(self): 숫자 데이터의 평균(산술 평균)을 계산합니다. 필요한 경우 _read_file을 호출하여 데이터가 사용 가능한지 확인합니다. 데이터가 없을 경우 ZeroDivisionError를 발생시킵니다.
힌트: 평균은 데이터의 합계를 항목 수로 나눈 값입니다. 나누기를 수행하기 전에 항목이 있는지 확인하여 0으로 나누는 오류를 방지하세요.

* min(self): 데이터셋에서 최소값을 계산하고 반환합니다. 아직 파일을 읽지 않았다면 파일을 읽어 데이터를 확인합니다. 데이터가 없을 경우 ValueError를 발생시킵니다.
힌트: Python의 내장 min() 함수를 사용하여 최소값을 찾습니다. 데이터가 없는 경우 적절하게 예외를 발생시켜야 합니다.

* max(self): 데이터셋에서 최대값을 계산하고 반환합니다. 필요한 경우 데이터를 읽고, 데이터가 비어 있으면 ValueError를 발생시킵니다.
힌트: calc_min()과 유사하게 구현되지만, max()를 사용하여 가장 큰 숫자를 반환합니다.

* median(self): 숫자 데이터의 중앙값을 계산하고 반환합니다. 데이터를 정렬한 후 중앙값을 계산합니다. 데이터 포인트 수가 홀수인 경우 중간값을 반환하고, 짝수인 경우 두 중간값의 평균을 반환합니다.
힌트: 데이터를 읽은 후 정렬하여 중앙값을 계산합니다. 데이터 길이가 짝수일 경우 중앙값은 두 중간 숫자의 평균이고, 홀수일 경우 중앙값은 중간 숫자입니다.

* save(self, output_filename):  모든 통계치(합계, 평균, 최소값, 최대값, 중앙값)를 계산하고, 이를 output_filename으로 지정된 새로운 파일에 저장합니다.

저장 예)

합계: 7.4<br>
평균: 2.5<br>
최소값: 1.5<br>
최대값: 3.2<br>
중앙값: 2.7<br>

## 실습 문제 4: csv 파일
다음을 계산하는 메서드를 포함한 CountryStatistics 클래스를 작성하세요.

파일: world-data-2023.csv
참고: https://gist.github.com/vr-23/d6a4a0aadcf3a2640091ca43c25e1955#file-world-data-2023-csv


### CountryStatistics 클래스

* clean_numeric_data(value: str): 문자열을 입력으로 받아 쉼표, 달러 기호, 백분율 기호를 제거하고 숫자 값을 float으로 반환합니다. 입력을 변환할 수 없으면 0.0을 반환합니다.
힌트: 이 함수는 쉼표나 달러 기호가 포함된 CSV 파일의 숫자 데이터를 처리하는 데 유용합니다. 빈 문자열이나 숫자가 아닌 값을 처리할 수 있도록 예외 처리에 신경 쓰세요.

* load_data(filename: str): CSV 파일에서 국가 데이터를 로드하고 데이터를 파싱하여 Country 객체를 생성합니다. 정리된 데이터는 self.countries 리스트에 저장됩니다. 파일 로드 완료 후 현재 시간을 self.load_time에 저장합니다.
힌트: csv.reader를 사용하여 CSV 파일을 한 줄씩 읽습니다. 첫 번째 행은 헤더이므로 건너뜁니다. 모든 숫자 데이터 필드에 대해 clean_numeric_data를 호출하여 데이터를 저장하기 전에 정확하게 정리된 데이터를 확인하세요. GDP, 기대 수명, 인구 등의 열 인덱스를 정확하게 매핑하세요.

* top_5_gdp(): GDP가 가장 높은 상위 5개 국가의 이름과 쉼표로 형식화된 GDP 값을 포함한 튜플 목록을 반환합니다.
힌트: 국가들을 gdp 속성 기준으로 내림차순 정렬하세요. 슬라이싱 ([:5])을 사용하여 상위 5개 항목을 선택하세요. 국가 이름과 형식화된 GDP 값을 반환하세요.

* top_5_life_expectancy(): 기대 수명이 가장 긴 상위 5개 국가의 이름과 그들의 기대 수명 값을 포함한 튜플 목록을 반환합니다.
힌트: 국가들을 기대 수명(life_expectancy) 기준으로 내림차순 정렬하세요. 슬라이싱 ([:5])을 사용하여 상위 5개 항목을 추출하세요. 기대 수명 값은 형식화할 필요가 없으므로 있는 그대로 반환하세요.

* top_5_density(): 인구 밀도가 가장 높은 상위 5개 국가의 이름과 인구 밀도를 반환합니다.
힌트: 국가들을 인구 밀도(density) 속성 기준으로 내림차순 정렬하세요. 슬라이싱 ([:5])을 사용하여 상위 5개 항목을 선택하세요. 인구 밀도 값은 정수이므로 형식화할 필요가 없습니다.

* get_data_load_time(format_str=None): 데이터 로드 시간을 반환하며, format_str로 사용자 지정 포맷을 지원합니다. 데이터가 아직 로드되지 않았다면 None을 반환합니다.
힌트: strftime 메서드를 이용해 봅니다.

* plot_geo_scatter(self): 위도와 경도 데이터를 사용하여 지리적 산점도를 그립니다.
힌트: plotly.express.scatter_geo 메서드를 이용해 봅니다. (16.2 이후 도전하기)


In [ ]:
import csv
from dataclasses import dataclass
import pandas as pd
import plotly.express as px

@dataclass
class Country:
    name: str
    gdp: float
    life_expectancy: float
    density: float
    lat: float
    lon: float

class CountryStatistics:
    # Implement your code